In [ ]:
!conda activate PythonData


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler,OneHotEncoder
#from keras.metrics import categorical_accuracy
#import tensorflow as tf
import glob
import os


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# merge the files
disease_path = "Disease_symptom_and_patient_profile_dataset.csv"

# Concatinate the files
disease_df = pd.read_csv(disease_path,encoding="utf-8")

# Review the DataFrame
disease_df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
344,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
346,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
347,Stroke,Yes,No,Yes,No,90,Female,High,High,Positive


In [ ]:
#identify the data types in the file before transforming
disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood Pressure        349 non-null    object
 8   Cholesterol Level     349 non-null    object
 9   Outcome Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB


In [ ]:
disease_df.nunique()

Disease                 116
Fever                     2
Cough                     2
Fatigue                   2
Difficulty Breathing      2
Age                      26
Gender                    2
Blood Pressure            3
Cholesterol Level         3
Outcome Variable          2
dtype: int64

In [ ]:
#initiate scaling
disease_data_scaled = StandardScaler().fit_transform(disease_df[["Age"]])
#create df with scaled data
disease_scaled_df = pd.DataFrame(disease_data_scaled,columns=["Age"])
#show the newly scaled df
disease_scaled_df.head()

,Age
0,-2.091160
1,-1.631964
2,-1.631964
3,-1.631964
4,-1.631964


In [ ]:
hot_df = pd.get_dummies(disease_df[['Outcome Variable','Gender','Difficulty Breathing','Fever','Cough','Fatigue','Blood Pressure','Cholesterol Level']])
hot_df

,Outcome Variable_Negative,Outcome Variable_Positive,Gender_Female,Gender_Male,Difficulty Breathing_No,Difficulty Breathing_Yes,Fever_No,Fever_Yes,Cough_No,Cough_Yes,Fatigue_No,Fatigue_Yes,Blood Pressure_High,Blood Pressure_Low,Blood Pressure_Normal,Cholesterol Level_High,Cholesterol Level_Low,Cholesterol Level_Normal
0,0,1,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,1
1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
2,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
3,0,1,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1
4,0,1,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0
345,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0
346,0,1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0
347,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0


In [ ]:
disease_concat_df = pd.concat([disease_df["Disease"],disease_scaled_df,hot_df],axis=1)
disease_concat_df.head()

,Disease,Age,Outcome Variable_Negative,Outcome Variable_Positive,Gender_Female,Gender_Male,Difficulty Breathing_No,Difficulty Breathing_Yes,Fever_No,Fever_Yes,Cough_No,Cough_Yes,Fatigue_No,Fatigue_Yes,Blood Pressure_High,Blood Pressure_Low,Blood Pressure_Normal,Cholesterol Level_High,Cholesterol Level_Low,Cholesterol Level_Normal
0,Influenza,-2.091160,0,1,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,1
1,Common Cold,-1.631964,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
2,Eczema,-1.631964,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
3,Asthma,-1.631964,0,1,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1
4,Asthma,-1.631964,0,1,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1


In [ ]:
y = disease_concat_df["Disease"].values
X = disease_concat_df.drop(["Disease"],1).values
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

/var/folders/3y/h60klfzn2s5d0bn8174h39z00000gn/T/ipykernel_1440/2889153076.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = disease_concat_df.drop(["Disease"],1).values


In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [ ]:
# Make a prediction using the testing data
test_predictions = logistic_regression_model.predict(X_test)
pd.DataFrame({'Predictions': test_predictions, 'Actual': y_test})

,Predictions,Actual
0,Tuberculosis,Hypothyroidism
1,Stroke,Tonsillitis
2,Stroke,Chronic Obstructive Pulmonary Disease (COPD)
3,Diabetes,Lyme Disease
4,Hypertension,Diabetes
...,...,...
83,Depression,Kidney Disease
84,Osteoporosis,Hypothyroidism
85,Kidney Cancer,Atherosclerosis
86,Stroke,Stroke


In [ ]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_test, test_predictions)}")

The balanced accuracy score of the model is: 0.0235042735042735


/Users/npatel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [ ]:
# Generate a confusion matrix for the model
cf_test_matrix = confusion_matrix(y_test, test_predictions)
cf_test_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 8, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# Print the classification report for the model
testing_report = classification_report(y_test, test_predictions)
print(testing_report)

                                              precision    recall  f1-score   support

                           Allergic Rhinitis       0.00      0.00      0.00         2
                         Alzheimer's Disease       0.00      0.00      0.00         1
                                      Asthma       0.50      0.89      0.64         9
                             Atherosclerosis       0.00      0.00      0.00         1
                              Bladder Cancer       0.00      0.00      0.00         1
                                 Brain Tumor       0.00      0.00      0.00         1
                               Breast Cancer       0.00      0.00      0.00         1
                                  Bronchitis       0.00      0.00      0.00         2
                                     Cholera       0.00      0.00      0.00         1
                      Chronic Kidney Disease       0.00      0.00      0.00         1
Chronic Obstructive Pulmonary Disease (COPD)       0.

/Users/npatel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/npatel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/npatel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/npatel/opt/anacon